In [6]:
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import Future 

In [7]:
import pandas as pd
import time
import requests
import os
from tqdm.auto import tqdm
import random
from bs4 import BeautifulSoup as bs


In [ ]:
pool = ProcessPoolExecutor(3) # This means a pool of 3 processes 


In [ ]:
df_en_ru_yandex = pd.read_csv("/Users/nigula/LL/adjust_unigr_dict/lookup results/yandex_lookup_en_ru_new_vs_pos_to_37353_no_freq.csv")


In [ ]:
def get_link(from_lang,to_lang, word):
    start = time.time()
    url = "https://context.reverso.net/перевод/" + from_lang + "-" + to_lang + "/" + word 
    return url



#     #print("going to find word in", url)
#     response = requests.get(url, headers=header_main, data = login)
#     response.encoding = 'utf-8' 
#     #
#     save_dir = os.path.join("/Users/nigula/LL/adjust_unigr_dict/lookup/reverso" + "_" + en_from_lang + "_" + en_to_lang,word + ".xls")
#     file = open(save_dir, "wb")
#     file.write(response.content)
#     file.close()
#     #print("saved to directory", save_dir)
#     time.sleep(random.uniform(0.1,1))
#     #print("request_time",time.time() - start) 


In [ ]:
v2_dict = pd.read_csv("./reverse_connected_dicts/to_release/merge_reverse_en_ru_fr_vs_ru_fr_en.csv")
words_to_be_handled_fr = list(set(v2_dict['fr']))
words_to_be_handled_en = list(set(v2_dict['en']))

In [ ]:
def get_links(df, lookup_from, lookup_to, target_lang, start_from_df_ind = None, finish_index = None):
    land_dict = {"русский":"ru","французский":"fr", "английский":"en"}
    count = 0 
    except_count = 0
    if start_from_df_ind == None:
        start = True
    else:
        start = False
    if finish_index == None:
        finish_index = len(df)
    url_list = []
    url_to_list = []
    handled_en = []
    habdled_local_word = []
    for index, row in tqdm(df.iterrows(), total=int(finish_index)):
        if str(count) == str(start_from_df_ind):
            start = True
        if str(count) == str(finish_index):
            print("FINISHED_ITERATE_COUNT")
            break
        if start == True:
            eng_word = row[land_dict[lookup_from]]
            local_word = row[land_dict[lookup_to]]
            
            #print(eng_word)
            if eng_word not in handled_en:
                url_curr = get_link(lookup_from, target_lang, eng_word)
                url_list.append(url_curr)
                handled_en.append(eng_word)
            if local_word not in habdled_local_word:
                url_curr = get_link(lookup_to, target_lang, local_word)
                url_to_list.append(url_curr)
                habdled_local_word.append(eng_word)
            
        count += 1
    return url_list, url_to_list

In [ ]:
url_list_fr_en,url_list_en_ru  = get_links(v2_dict, "французский", "английский", "русский")

In [ ]:
en_ru_fr_dict = pd.read_csv("/Users/nigula/LL/adjust_unigr_dict/reverse_connected_dicts/to_release/en_ru_fr_V1.csv")
russian_words = list(set(en_ru_fr_dict['ru']))

In [ ]:
url_list_fr_en,url_list_ru_en  = get_links(en_ru_fr_dict, "французский", "русский", "английский")

In [ ]:
url_list_en_fr,url_list_ru_fr  = get_multilang_from_df(df_en_ru_yandex, "английский", "русский", "французский", 
                                                       start_from_df_ind= 45000,
                                                      finish_index = 120000)

In [ ]:
print(len(url_list_ru_fr))
url_list_ru_fr[:10]

In [ ]:
url_list_en_fr.index("https://context.reverso.net/перевод/английский-французский/reservoir")

In [ ]:
print(len(url_list_en_fr))
url_list_en_fr[:10]

In [8]:
infile = open("abuse_example.xls","r")
contents = infile.read()
soup_out = bs(contents,'html.parser')

In [9]:
def check_restiction(soup):
    for link in soup.find_all("section", attrs={"id" : "error-content"}):
        restrict = ['Your', 'access', 'is', 'temporarily', 'restricted']
        #print(link, "\\")
        for l in link.find_all("p"):
            count = 0 
            restict = False
            for word in l.text.strip().split()[:5]:
                #print(word, restrict[count])
                if word == restrict[count]:
                    restict = True
                else:
                    restict = False
                count += 1
            if restict == True:
                print("restcit_eba")
                raise Exception('Restiction_limit!!!!!!!!!!!!!!')

check_restiction(soup_out)

restcit_eba


Exception: Restiction_limit!!!!!!!!!!!!!!

In [ ]:
ur = 'https://context.reverso.net/перевод/русский-французский/заросли'
ur.split("/")[-1]

In [10]:
def only_send_request_func(url):
    #print("here")
    land_dict = {"русский":"ru","французский":"fr", "английский":"en"}
    lang_pair = url.split("/")[-2]
    from_lang = lang_pair.split("-")[0]
    to_lang = lang_pair.split("-")[1]
    save_loc_base = "/Users/nigula/LL/adjust_unigr_dict/lookup/reverso_" + land_dict[from_lang] + "_" + land_dict[to_lang]
    handled_words_list = []
    for word in os.listdir(save_loc_base):
        handled_words_list.append(word.split(".")[0])
    handled_words_set = set(handled_words_list)   
    word_to_be_requested = url.split("/")[-1]
    #print("extract word")
    if word_to_be_requested not in handled_words_set:
        #print("to be handled")
        login = {'inUserName': 'n.babakov@lingualeo.com', 'inUserPass': '33vec33'}
        header_main = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
        start = time.time()
        response = requests.get(url, headers=header_main, data = login)
        response.encoding = 'utf-8' 
        soup = bs(response.text, 'html.parser') 
        is_restricted = check_restiction(soup)
        save_dir = os.path.join(save_loc_base ,url.split("/")[-1] + ".xls")
        #print("saved at ",save_dir)
        file = open(save_dir, "wb")
        file.write(response.content)
        file.close()
        time.sleep(random.uniform(0.1, 1))
    else:
        #print(word_to_be_requested, "already_handles")
        pass
only_send_request_func('https://context.reverso.net/перевод/русский-французский/мама')


In [ ]:
url_list_fr_en[:10]

In [11]:
login = {'inUserName': 'n.babakov@lingualeo.com', 'inUserPass': '33vec33'}
header_main = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
only_send_request_func("https://context.reverso.net/перевод/французский-английский/joyeusement")


In [12]:
from concurrent.futures import ProcessPoolExecutor, as_completed

In [13]:
len(url_list_fr_en)

NameError: name 'url_list_fr_en' is not defined

In [14]:
import json

In [2]:
with open("url_list_en_ru.json" , "r") as f:
    url_list_en_ru = json.load(f)

In [3]:
url_list_en_ru[:10]

['https://context.reverso.net/перевод/английский-русский/gaily',
 'https://context.reverso.net/перевод/английский-русский/merrily',
 'https://context.reverso.net/перевод/английский-русский/blithely',
 'https://context.reverso.net/перевод/английский-русский/cheerily',
 'https://context.reverso.net/перевод/английский-русский/jolly',
 'https://context.reverso.net/перевод/английский-русский/happy',
 'https://context.reverso.net/перевод/английский-русский/cheerful',
 'https://context.reverso.net/перевод/английский-русский/jolly',
 'https://context.reverso.net/перевод/английский-русский/jauntily',
 'https://context.reverso.net/перевод/английский-русский/humorously']

In [ ]:
with open("url_list_en_ru.json" , "w") as f:
    json.dump(url_list_en_ru, f, indent = 1)

In [ ]:
url_list_fr_en[:10]

In [ ]:
with open("url_list_fr_ru.json" , "w") as f:
    json.dump(url_list_fr_en, f, indent = 1)

In [16]:
list_to_be_processed = url_list_en_ru
from_ind = 0
to_ind = len(list_to_be_processed)

with ProcessPoolExecutor(max_workers=2) as executor:
    start = time.time()
    futures = [ executor.submit(only_send_request_func, url) for url in list_to_be_processed[from_ind:to_ind]]
    results = []
    for result in tqdm(as_completed(futures), total=int(to_ind-from_ind)):
        results.append(result)
    end = time.time()
    print("Time Taken: {:.6f}s".format(end-start))


Time Taken: 33.838780s
